In [1]:
import tensorflow
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.layers import Embedding, LSTM, Dense
from tensorflow.keras.models import Sequential
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import Adam
import pickle
import numpy as np
import os

In [2]:
os.getcwd()

'C:\\Users\\DELL'

In [3]:
os.chdir("F:/Auto completion/AUTO_COMPLETE_STREAMLIT-master/DATA")

In [4]:
os.getcwd()

'F:\\Auto completion\\AUTO_COMPLETE_STREAMLIT-master\\DATA'

In [5]:
with  open('data.txt','r',encoding='utf-8') as f:
    file=f.read()

In [6]:
len(file)

148201

In [7]:
print(file[:100])

   GOOD WILL HUNTING
           by
Matt Damon & Ben Affleck

FADE IN:
EXT. SOUTH BOSTON ST. PATRICK'


In [8]:
#store files in list
#lines=[]
#for i in file:
    #lines.append(i)
lines = file.split('\n')
#convert list to string
data=' '.join(lines)
#replace unnecessary stuff with space
data=data.replace('\n', '').replace('\r', '').replace('\ufeff', '').replace('"', ' ').replace('--', ' ')  #new line, carriage return, unicode character, opening and closing double quotes with spaces
#remove unnecessary spaces
data=data.split()
data=' '.join(data)
data[:500]

"GOOD WILL HUNTING by Matt Damon & Ben Affleck FADE IN: EXT. SOUTH BOSTON ST. PATRICK'S DAY PARADE DAY CUT TO: INT. L STREET BAR & GRILLE, SOUTH BOSTON EVENING The bar is dirty, more than a little run down. If there is ever a cook on duty, he's not here now. As we pan across several empty tables, we can almost smell the odor of last nights beer and crushed pretzels on the floor. CHUCKIE Oh my God, I got the most fucked up thing I been meanin' to tell you. As the camera rises, we find FOUR YOUNG M"

## Applying Tokenization

In [9]:
tokenizer=Tokenizer()
tokenizer.fit_on_texts([data])
#saving tokenizer for predict function
pickle.dump(tokenizer,open('token.pkl','wb'))

sequence_data=tokenizer.texts_to_sequences([data])[0]
sequence_data[:15]

[96, 2, 645, 90, 1527, 1528, 1529, 1530, 1024, 12, 77, 225, 163, 1025, 1531]

In [10]:
len(sequence_data)

23607

In [11]:
sequences=[]
for i in range(3,len(sequence_data)):
    words=sequence_data[i-3:i+1]
    sequences.append(words)
    
print("the length of sequences are:",len(sequences))
sequences=np.array(sequences)
sequences[:10]

the length of sequences are: 23604


array([[  96,    2,  645,   90],
       [   2,  645,   90, 1527],
       [ 645,   90, 1527, 1528],
       [  90, 1527, 1528, 1529],
       [1527, 1528, 1529, 1530],
       [1528, 1529, 1530, 1024],
       [1529, 1530, 1024,   12],
       [1530, 1024,   12,   77],
       [1024,   12,   77,  225],
       [  12,   77,  225,  163]])

In [13]:
X=[]
y=[]

for i in sequences:
    X.append(i[0:3])
    y.append(i[3])
    
X=np.array(X)
y=np.array(y)

In [14]:
print("Data: ",X[:10])
print("Response: ",y[:10])

Data:  [[  96    2  645]
 [   2  645   90]
 [ 645   90 1527]
 [  90 1527 1528]
 [1527 1528 1529]
 [1528 1529 1530]
 [1529 1530 1024]
 [1530 1024   12]
 [1024   12   77]
 [  12   77  225]]
Response:  [  90 1527 1528 1529 1530 1024   12   77  225  163]


In [16]:
vocab_size=len(tokenizer.word_index)+1
print(vocab_size)

3293


In [17]:
y=to_categorical(y,num_classes=vocab_size)
y[:5]

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)

## Creating Model

In [18]:
model=Sequential()
model.add(Embedding(vocab_size,10,input_length=3))
model.add(LSTM(1000,return_sequences=True))
model.add(LSTM(1000))
model.add(Dense(1000,activation="relu"))
model.add(Dense(vocab_size,activation="softmax"))

In [20]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 3, 10)             32930     
                                                                 
 lstm (LSTM)                 (None, 3, 1000)           4044000   
                                                                 
 lstm_1 (LSTM)               (None, 1000)              8004000   
                                                                 
 dense (Dense)               (None, 1000)              1001000   
                                                                 
 dense_1 (Dense)             (None, 3293)              3296293   
                                                                 
Total params: 16,378,223
Trainable params: 16,378,223
Non-trainable params: 0
_________________________________________________________________


In [21]:
from tensorflow.keras.callbacks import ModelCheckpoint

checkpoint=ModelCheckpoint("next_words.h5",monitor="loss",verbose=1,save_best_only=True)
model.compile(loss="categorical_crossentropy",optimizer=Adam(learning_rate=0.001))
model.fit(X,y,epochs=70,batch_size=64,callbacks=[checkpoint])

Epoch 1/70
369/369 [==============================] - ETA: 0s - loss: 6.5654
Epoch 1: loss improved from inf to 6.56543, saving model to next_words.h5
369/369 [==============================] - 73s 189ms/step - loss: 6.5654
Epoch 2/70
369/369 [==============================] - ETA: 0s - loss: 6.1344
Epoch 2: loss improved from 6.56543 to 6.13436, saving model to next_words.h5
369/369 [==============================] - 69s 188ms/step - loss: 6.1344
Epoch 3/70
369/369 [==============================] - ETA: 0s - loss: 5.8316
Epoch 3: loss improved from 6.13436 to 5.83163, saving model to next_words.h5
369/369 [==============================] - 69s 187ms/step - loss: 5.8316
Epoch 4/70
369/369 [==============================] - ETA: 0s - loss: 5.5672
Epoch 4: loss improved from 5.83163 to 5.56716, saving model to next_words.h5
369/369 [==============================] - 69s 187ms/step - loss: 5.5672
Epoch 5/70
369/369 [==============================] - ETA: 0s - loss: 5.3183
Epoch 5: loss i

Epoch 37/70
369/369 [==============================] - ETA: 0s - loss: 0.3256
Epoch 37: loss improved from 0.33513 to 0.32556, saving model to next_words.h5
369/369 [==============================] - 70s 191ms/step - loss: 0.3256
Epoch 38/70
369/369 [==============================] - ETA: 0s - loss: 0.3294
Epoch 38: loss did not improve from 0.32556
369/369 [==============================] - 71s 192ms/step - loss: 0.3294
Epoch 39/70
369/369 [==============================] - ETA: 0s - loss: 0.3217
Epoch 39: loss improved from 0.32556 to 0.32169, saving model to next_words.h5
369/369 [==============================] - 71s 192ms/step - loss: 0.3217
Epoch 40/70
369/369 [==============================] - ETA: 0s - loss: 0.3136
Epoch 40: loss improved from 0.32169 to 0.31358, saving model to next_words.h5
369/369 [==============================] - 70s 191ms/step - loss: 0.3136
Epoch 41/70
369/369 [==============================] - ETA: 0s - loss: 0.3215
Epoch 41: loss did not improve from 0

## prediction

In [23]:
from tensorflow.keras.models import load_model
import numpy as np
import pickle

In [25]:
#load model and tokenizer
model=load_model('next_words.h5')
tokenizer=pickle.load(open('token.pkl','rb'))

def Predict_Next_Words(model,tokenizer,text):
    sequence=tokenizer.texts_to_sequences([text])
    sequence=np.array(sequence)
    preds=np.argmax(model.predict(sequence))
    predicted_word=""

    for key,value in tokenizer.word_index.items():
        if value==preds:
            predicted_word=key
            break
    print(predicted_word)
    return predicted_word

    

In [26]:
while(True):
    text=input("Enter you line: ")
    
    if text=="0":
        print("Execution completed...")
        break
    else:
        try:
            text=text.split(" ")
            text=text[-3:]
            print(text)
            Predict_Next_Words(model,tokenizer,text)
        except Exception as e:
            print("Error occured: ")
            continue

Enter you line: As the camera
['As', 'the', 'camera']
1/1 [==============================] - 1s 650ms/step
rises
Enter you line: 
['']
Error occured: 
Enter you line: 
['']
Error occured: 
Enter you line: hi
['hi']
1/1 [==============================] - 1s 571ms/step
i
Enter you line: 0
Execution completed...
